# Understand dag in spark web ui

In this section, we will learn how to read the DAG in the spark web ui

As you know, spark uses lazy execution model, then you run a transformation, there will be no actual job that execute the transformation. A **spark job** is triggered until you execute an action (e.g. show, collect, take, count, etc.)


When a spark job is executed, In spark web UI (Jobs tab), you will find the job grouped by its status (e.g. running, succeed, failed.). Click on the job description, you will land on the **details for job** page. In this page, you can view the execution DAG of all stages. **A DAG represents a chain of RDD transformations**

In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col

In [2]:
local = True

if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("UnderstandSparkUIDAG") \
        .getOrCreate()
else:
    spark = SparkSession.builder \
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("UnderstandSparkUIDAG") \
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:py3.9.7-spark3.2.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "8g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

22/03/02 08:58:46 WARN Utils: Your hostname, pliu-SATELLITE-P850 resolves to a loopback address: 127.0.1.1; using 172.22.0.33 instead (on interface wlp3s0)
22/03/02 08:58:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 08:58:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df=spark.range(1,100000)
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
+---+
only showing top 20 rows



In [6]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


After running the above code, open your spark UI (if you run in local mode, the default url is http://localhost:4040/jobs/). You should see a job, click on it, you should see a page like below figure
![Spark_UI_JOB_details](../img/spark_ui_job_details.png)

In the **event timeline**, you can check the lifecycle of each executor (e.g. added, removed) and stages of this job. Executors are per application, when an application is terminated, all executors belongs to this application will be removed. In cluster mode, each executor is an individual jvm process.

In the **DAG Visualization**, the **blue shaded boxes represent to the Spark operation** that the user calls in the code. The **dots in these boxes represent RDDs created in the corresponding operations**.


In our case, we have two boxes. The name of the first box is **WholeStageCodegen**, because we create a dataframe that involves java code generation to build underlying RDDs. Then second box is **mapPartitionsInternal**, because show




## Spark DAG stage operation name
- **WholeStageCodegen**: happens when you run computations on DataFrames and generates Java code to build underlying RDDs

In [4]:
df.cache()

DataFrame[id: bigint]

In [5]:
df.select((col("id")*5).alias("id")).show()

+---+
| id|
+---+
|  5|
| 10|
| 15|
| 20|
| 25|
| 30|
| 35|
| 40|
| 45|
| 50|
| 55|
| 60|
| 65|
| 70|
| 75|
| 80|
| 85|
| 90|
| 95|
|100|
+---+
only showing top 20 rows

